In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain

def get_documents_from_web(url):
    loader = WebBaseLoader(url)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=20
    )
    splitDocs = splitter.split_documents(docs)
    return splitDocs

def create_db(docs):
    embedding = OpenAIEmbeddings()
    vectorStore = FAISS.from_documents(docs, embedding = embedding)
    return vectorStore

def create_chain(vectorStore):
    # 初始化 OllamaLLM
    model = OllamaLLM(model='phi3') 

    # 定義 Prompt 模板
    prompt = PromptTemplate.from_template("""
    Answer the user's question:
    context: {context}
    Question: {input}
    """)

    #chain = prompt | model
    chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt
    )

    retrieval = vectorStore.as_retrieval(search_kwargs={"k": 3})
    retrieval_chain = create_retrieval_chain(retrieval,chain)
    
    return retrieval_chain

docs =get_documents_from_web("https://python.langchain.com/v0.1/docs/expression_language/")
vectorStore = create_db(docs)
chain = create_chain(vectorStore)

response = chain.invoke({
    "input" : "What is LCEL?",
})
print(response["answer"])

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable